#  NFL Big Data Bowl 2026 - LB 0.604 Pipeline
## **Advanced Player Trajectory Prediction with Hybrid Sequence Models**
## This notebook implements the complete inference pipeline from my LB 0.604 solution, featuring:
### - 114 comprehensive features with player interactions
### - 5-fold ensemble of hybrid GRU+Transformer models
### - Real-time sequence processing for competition inference
#
---

# IMPORT & CONFIG

In [ ]:
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
from pathlib import Path
import joblib
import os
import polars as pl
from sklearn.preprocessing import StandardScaler
import kaggle_evaluation.nfl_inference_server

class Config:
    """Configuration class for model and data parameters"""
    # Paths
    DATA_DIR = Path("/kaggle/input/nfl-big-data-bowl-2026-prediction/")
    MODEL_DIR = Path("/kaggle/input/nfl-big-data-bowl-2026-lb-0-604/")
    
    # Model parameters
    SEED = 42
    WINDOW_SIZE = 12
    HIDDEN_DIM = 192
    MAX_FUTURE_HORIZON = 94
    BATCH_SIZE = 512
    
    # Feature engineering
    USE_PLAYERS_INTERACTIONS = True
    FIELD_X_MIN, FIELD_X_MAX = 0.0, 120.0
    FIELD_Y_MIN, FIELD_Y_MAX = 0.0, 53.3
    
    # Hardware
    DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    # Feature groups (for organization)
    BASIC_FEATURES = ['x', 'y', 's', 'a', 'o', 'dir']
    PHYSICS_FEATURES = ['velocity_x', 'velocity_y', 'acceleration_x', 'acceleration_y']
    ROLE_FEATURES = ['is_offense', 'is_defense', 'is_receiver', 'is_coverage', 'is_passer']
    TEMPORAL_FEATURES = ['x_lag1', 'y_lag1', 'velocity_x_lag1', 'velocity_y_lag1']

# Set random seeds for reproducibility
def set_seed(seed: int = Config.SEED):
    """Set random seeds for reproducibility"""
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)

set_seed()

# MODEL ARCHITECTURE - Same as training


In [ ]:
class HybridSeqModel(nn.Module):
    """
    Hybrid GRU + Conv1D + Transformer model for player trajectory prediction
    This matches exactly the architecture used in training
    """
    def __init__(self, input_dim, horizon):
        super().__init__()
        self.horizon = horizon
        self.gru = nn.GRU(input_dim, 192, num_layers=3, batch_first=True, dropout=0.2, bidirectional=False)
        self.conv1d = nn.Sequential(
            nn.Conv1d(192, 128, kernel_size=3, padding=1),
            nn.GELU(),
            nn.Dropout(0.1),
            nn.Conv1d(128, 128, kernel_size=5, padding=2),
            nn.GELU(),
        )
        self.pool_ln = nn.LayerNorm(192)
        self.pool_attn = nn.MultiheadAttention(192, num_heads=8, batch_first=True, dropout=0.1)
        self.pool_query = nn.Parameter(torch.randn(1, 1, 192))
        self.head_shared = nn.Sequential(
            nn.Linear(192 + 128, 256),
            nn.GELU(),
            nn.Dropout(0.3),
            nn.Linear(256, 128),
            nn.GELU(),
            nn.Dropout(0.2),
        )
        self.head_x = nn.Linear(128, horizon)
        self.head_y = nn.Linear(128, horizon)
    
    def forward(self, x):
        h, _ = self.gru(x)
        h_conv = self.conv1d(h.transpose(1, 2)).transpose(1, 2)
        h_conv_pool = h_conv.mean(dim=1)
        B = h.size(0)
        q = self.pool_query.expand(B, -1, -1)
        h_norm = self.pool_ln(h)
        ctx, _ = self.pool_attn(q, h_norm, h_norm)
        ctx = ctx.squeeze(1)
        combined = torch.cat([ctx, h_conv_pool], dim=1)
        shared = self.head_shared(combined)
        out_x = torch.cumsum(self.head_x(shared), dim=1)
        out_y = torch.cumsum(self.head_y(shared), dim=1)
        return out_x, out_y

# FEATURE ENGINEERING PIPELINE

In [ ]:
class CompleteFeatureEngineer:
    def __init__(self):
        pass
    
    @staticmethod
    def height_to_feet(height_str):
        try:
            ft, inches = map(int, str(height_str).split('-'))
            return ft + inches/12
        except:
            return 6.0
    
    def add_advanced_features(self, df):
        """Enhanced feature engineering from training"""
        df = df.copy()
        df = df.sort_values(['game_id', 'play_id', 'nfl_id', 'frame_id'])
        gcols = ['game_id', 'play_id', 'nfl_id']
        
        # Distance Rate Features
        if 'distance_to_ball' in df.columns:
            df['distance_to_ball_change'] = df.groupby(gcols)['distance_to_ball'].diff().fillna(0)
            df['distance_to_ball_accel'] = df.groupby(gcols)['distance_to_ball_change'].diff().fillna(0)
            df['time_to_intercept'] = (df['distance_to_ball'] / 
                                        (np.abs(df['distance_to_ball_change']) + 0.1)).clip(0, 10).fillna(0)
        
        # Target Alignment Features
        if 'ball_direction_x' in df.columns:
            df['velocity_alignment'] = (
                df['velocity_x'] * df['ball_direction_x'] +
                df['velocity_y'] * df['ball_direction_y']
            ).fillna(0)
            df['velocity_perpendicular'] = (
                df['velocity_x'] * (-df['ball_direction_y']) +
                df['velocity_y'] * df['ball_direction_x']
            ).fillna(0)
            if 'acceleration_x' in df.columns:
                df['accel_alignment'] = (
                    df['acceleration_x'] * df['ball_direction_x'] +
                    df['acceleration_y'] * df['ball_direction_y']
                ).fillna(0)
        
        # Multi-Window Rolling
        for window in [3, 5, 10]:
            for col in ['velocity_x', 'velocity_y', 's', 'a']:
                if col in df.columns:
                    df[f'{col}_roll{window}'] = df.groupby(gcols)[col].transform(
                        lambda x: x.rolling(window, min_periods=1).mean()
                    ).fillna(0)
                    df[f'{col}_std{window}'] = df.groupby(gcols)[col].transform(
                        lambda x: x.rolling(window, min_periods=1).std()
                    ).fillna(0)
        
        # Extended Lag Features
        for lag in [4, 5]:
            for col in ['x', 'y', 'velocity_x', 'velocity_y']:
                if col in df.columns:
                    df[f'{col}_lag{lag}'] = df.groupby(gcols)[col].shift(lag).fillna(0)
        
        # Velocity Change Features
        if 'velocity_x' in df.columns:
            df['velocity_x_change'] = df.groupby(gcols)['velocity_x'].diff().fillna(0)
            df['velocity_y_change'] = df.groupby(gcols)['velocity_y'].diff().fillna(0)
            df['speed_change'] = df.groupby(gcols)['s'].diff().fillna(0)
            df['direction_change'] = df.groupby(gcols)['dir'].diff().fillna(0)
            df['direction_change'] = df['direction_change'].apply(
                lambda x: x if abs(x) < 180 else x - 360 * np.sign(x) if not pd.isna(x) else 0
            )
        
        # Field Position Features
        df['dist_from_left'] = df['y']
        df['dist_from_right'] = 53.3 - df['y']
        df['dist_from_sideline'] = np.minimum(df['dist_from_left'], df['dist_from_right'])
        df['dist_from_endzone'] = np.minimum(df['x'], 120 - df['x'])
        
        # Role-Specific Features
        if 'is_receiver' in df.columns and 'velocity_alignment' in df.columns:
            df['receiver_optimality'] = df['is_receiver'] * df['velocity_alignment']
            df['receiver_deviation'] = df['is_receiver'] * np.abs(df.get('velocity_perpendicular', 0))
        if 'is_coverage' in df.columns and 'closing_speed' in df.columns:
            df['defender_closing_speed'] = df['is_coverage'] * df['closing_speed']
        
        # Time Features
        df['frames_elapsed'] = df.groupby(gcols).cumcount()
        df['normalized_time'] = df.groupby(gcols)['frames_elapsed'].transform(
            lambda x: x / (x.max() + 1)
        ).fillna(0)
        
        return df

    def compute_player_interactions_fast(self, df):
        """Fast version of player interactions for inference"""
        print("Computing player interactions...")
        
        agg_rows = []
        for (g, p, f), grp in df.groupby(['game_id', 'play_id', 'frame_id']):
            n = len(grp)
            if n < 2:
                continue
                
            x = grp['x'].to_numpy(dtype=np.float32)
            y = grp['y'].to_numpy(dtype=np.float32)
            vx = grp['velocity_x'].to_numpy(dtype=np.float32)
            vy = grp['velocity_y'].to_numpy(dtype=np.float32)
            is_off = grp['is_offense'].to_numpy().astype(bool)
            nfl_ids = grp['nfl_id'].to_numpy()
            
            # Pairwise geometry
            dx = x[None, :] - x[:, None]
            dy = y[None, :] - y[:, None]
            dist = np.sqrt(dx * dx + dy * dy)
            angle_mat = np.arctan2(-dy, -dx)
            dvx = vx[:, None] - vx[None, :]
            dvy = vy[:, None] - vy[None, :]
            rel_speed = np.sqrt(dvx * dvx + dvy * dvy)
            
            # Masks
            opp_mask = (is_off[:, None] != is_off[None, :])
            np.fill_diagonal(opp_mask, False)
            
            mask_off = np.broadcast_to(is_off[None, :], (n, n)).copy()
            mask_def = np.broadcast_to(~is_off[None, :], (n, n)).copy()
            np.fill_diagonal(mask_off, False)
            np.fill_diagonal(mask_def, False)
            
            # Nearest opponent
            dist_opp = np.where(opp_mask, dist, np.nan)
            nearest_dist = np.nanmin(dist_opp, axis=1)
            nearest_idx = np.nanargmin(dist_opp, axis=1)
            all_nan = ~np.isfinite(nearest_dist)
            nearest_idx_safe = nearest_idx.copy()
            nearest_idx_safe[all_nan] = 0
            nearest_angle = np.take_along_axis(angle_mat, nearest_idx_safe[:, None], axis=1).squeeze(1)
            nearest_rel = np.take_along_axis(rel_speed, nearest_idx_safe[:, None], axis=1).squeeze(1)
            nearest_angle[all_nan] = np.nan
            nearest_rel[all_nan] = np.nan
            
            # Group-wise aggregations
            d_off = np.where(mask_off, dist, np.nan)
            d_def = np.where(mask_def, dist, np.nan)
            d_mean_o = np.nanmean(d_off, axis=1); d_min_o = np.nanmin(d_off, axis=1); d_max_o = np.nanmax(d_off, axis=1)
            d_mean_d = np.nanmean(d_def, axis=1); d_min_d = np.nanmin(d_def, axis=1); d_max_d = np.nanmax(d_def, axis=1)
            
            v_off = np.where(mask_off, rel_speed, np.nan)
            v_def = np.where(mask_def, rel_speed, np.nan)
            v_mean_o = np.nanmean(v_off, axis=1); v_min_o = np.nanmin(v_off, axis=1); v_max_o = np.nanmax(v_off, axis=1)
            v_mean_d = np.nanmean(v_def, axis=1); v_min_d = np.nanmin(v_def, axis=1); v_max_d = np.nanmax(v_def, axis=1)
            
            sinA = np.sin(angle_mat); cosA = np.cos(angle_mat)
            cnt_off = mask_off.sum(axis=1).astype(np.float32)
            cnt_def = mask_def.sum(axis=1).astype(np.float32)
            denom_off = np.where(cnt_off > 0, cnt_off, np.nan)
            denom_def = np.where(cnt_def > 0, cnt_def, np.nan)
            
            sin_sum_off = (sinA * mask_off).sum(axis=1)
            cos_sum_off = (cosA * mask_off).sum(axis=1)
            sin_sum_def = (sinA * mask_def).sum(axis=1)
            cos_sum_def = (cosA * mask_def).sum(axis=1)
            
            a_mean_o = np.arctan2(sin_sum_off / denom_off, cos_sum_off / denom_off)
            a_mean_d = np.arctan2(sin_sum_def / denom_def, cos_sum_def / denom_def)
            
            a_off = np.where(mask_off, angle_mat, np.nan)
            a_def = np.where(mask_def, angle_mat, np.nan)
            a_min_o = np.nanmin(a_off, axis=1); a_max_o = np.nanmax(a_off, axis=1)
            a_min_d = np.nanmin(a_def, axis=1); a_max_d = np.nanmax(a_def, axis=1)
            
            for idx, nid in enumerate(nfl_ids):
                agg_rows.append({
                    'game_id': g, 'play_id': p, 'frame_id': f, 'nfl_id': int(nid),
                    'distance_to_player_mean_offense': d_mean_o[idx],
                    'distance_to_player_min_offense': d_min_o[idx],
                    'distance_to_player_max_offense': d_max_o[idx],
                    'relative_velocity_magnitude_mean_offense': v_mean_o[idx],
                    'relative_velocity_magnitude_min_offense': v_min_o[idx],
                    'relative_velocity_magnitude_max_offense': v_max_o[idx],
                    'angle_to_player_mean_offense': a_mean_o[idx],
                    'angle_to_player_min_offense': a_min_o[idx],
                    'angle_to_player_max_offense': a_max_o[idx],
                    'distance_to_player_mean_defense': d_mean_d[idx],
                    'distance_to_player_min_defense': d_min_d[idx],
                    'distance_to_player_max_defense': d_max_d[idx],
                    'relative_velocity_magnitude_mean_defense': v_mean_d[idx],
                    'relative_velocity_magnitude_min_defense': v_min_d[idx],
                    'relative_velocity_magnitude_max_defense': v_max_d[idx],
                    'angle_to_player_mean_defense': a_mean_d[idx],
                    'angle_to_player_min_defense': a_min_d[idx],
                    'angle_to_player_max_defense': a_max_d[idx],
                    'nearest_opponent_dist': float(nearest_dist[idx]) if np.isfinite(nearest_dist[idx]) else 0.0,
                    'nearest_opponent_angle': float(nearest_angle[idx]) if np.isfinite(nearest_angle[idx]) else 0.0,
                    'nearest_opponent_rel_speed': float(nearest_rel[idx]) if np.isfinite(nearest_rel[idx]) else 0.0,
                })
        
        return pd.DataFrame(agg_rows) if agg_rows else pd.DataFrame()

    def prepare_sequences_complete(self, input_df, test_template=None, is_training=False, window_size=12):
        """Complete sequence preparation matching training pipeline"""
        print("Preparing sequences with complete 114-feature engineering...")
        
        input_df = input_df.copy()
        
        # Step 1: Basic features
        print("Step 1/4: Adding basic features...")
        input_df['player_height_feet'] = input_df['player_height'].apply(self.height_to_feet)
        
        dir_rad = np.deg2rad(input_df['dir'].fillna(0))
        delta_t = 0.1
        input_df['velocity_x'] = (input_df['s'] + 0.5 * input_df['a'] * delta_t) * np.sin(dir_rad)
        input_df['velocity_y'] = (input_df['s'] + 0.5 * input_df['a'] * delta_t) * np.cos(dir_rad)
        input_df['acceleration_x'] = input_df['a'] * np.sin(dir_rad)
        input_df['acceleration_y'] = input_df['a'] * np.cos(dir_rad)
        input_df['o_sin'] = np.sin(np.deg2rad(input_df['o'].fillna(0)))
        input_df['o_cos'] = np.cos(np.deg2rad(input_df['o'].fillna(0)))
        input_df['dir_sin'] = np.sin(np.deg2rad(input_df['dir'].fillna(0)))
        input_df['dir_cos'] = np.cos(np.deg2rad(input_df['dir'].fillna(0)))
        
        # Roles
        input_df['is_offense'] = (input_df['player_side'] == 'Offense').astype(int)
        input_df['is_defense'] = (input_df['player_side'] == 'Defense').astype(int)
        input_df['is_receiver'] = (input_df['player_role'] == 'Targeted Receiver').astype(int)
        input_df['is_coverage'] = (input_df['player_role'] == 'Defensive Coverage').astype(int)
        input_df['is_passer'] = (input_df['player_role'] == 'Passer').astype(int)
        
        # Physics
        mass_kg = input_df['player_weight'].fillna(200.0) / 2.20462
        input_df['momentum_x'] = input_df['velocity_x'] * mass_kg
        input_df['momentum_y'] = input_df['velocity_y'] * mass_kg
        input_df['kinetic_energy'] = 0.5 * mass_kg * (input_df['s'] ** 2)
        
        # Ball features
        if 'ball_land_x' in input_df.columns:
            ball_dx = input_df['ball_land_x'] - input_df['x']
            ball_dy = input_df['ball_land_y'] - input_df['y']
            input_df['distance_to_ball'] = np.sqrt(ball_dx**2 + ball_dy**2)
            input_df['angle_to_ball'] = np.arctan2(ball_dy, ball_dx)
            input_df['ball_direction_x'] = ball_dx / (input_df['distance_to_ball'] + 1e-6)
            input_df['ball_direction_y'] = ball_dy / (input_df['distance_to_ball'] + 1e-6)
            input_df['closing_speed'] = (
                input_df['velocity_x'] * input_df['ball_direction_x'] +
                input_df['velocity_y'] * input_df['ball_direction_y']
            )
        
        # Sort for temporal
        input_df = input_df.sort_values(['game_id', 'play_id', 'nfl_id', 'frame_id'])
        gcols = ['game_id', 'play_id', 'nfl_id']
        
        # Lag features
        for lag in [1, 2, 3]:
            input_df[f'x_lag{lag}'] = input_df.groupby(gcols)['x'].shift(lag).fillna(0)
            input_df[f'y_lag{lag}'] = input_df.groupby(gcols)['y'].shift(lag).fillna(0)
            input_df[f'velocity_x_lag{lag}'] = input_df.groupby(gcols)['velocity_x'].shift(lag).fillna(0)
            input_df[f'velocity_y_lag{lag}'] = input_df.groupby(gcols)['velocity_y'].shift(lag).fillna(0)
        
        # EMA features
        input_df['velocity_x_ema'] = input_df.groupby(gcols)['velocity_x'].transform(
            lambda x: x.ewm(alpha=0.3, adjust=False).mean()
        ).fillna(0)
        input_df['velocity_y_ema'] = input_df.groupby(gcols)['velocity_y'].transform(
            lambda x: x.ewm(alpha=0.3, adjust=False).mean()
        ).fillna(0)
        input_df['speed_ema'] = input_df.groupby(gcols)['s'].transform(
            lambda x: x.ewm(alpha=0.3, adjust=False).mean()
        ).fillna(0)
        
        # Step 2: Advanced features
        print("Step 2/4: Adding advanced features...")
        input_df = self.add_advanced_features(input_df)
        
        # Step 3: Player interactions
        print("Step 3/4: Adding player interaction features...")
        if Config.USE_PLAYERS_INTERACTIONS:
            interaction_agg = self.compute_player_interactions_fast(input_df)
            if not interaction_agg.empty:
                input_df = input_df.merge(
                    interaction_agg,
                    on=['game_id', 'play_id', 'frame_id', 'nfl_id'],
                    how='left'
                )
        
        # Step 4: Create sequences
        print("Step 4/4: Creating sequences...")
        
        # Complete feature list (114 features)
        feature_cols = [
            'x', 'y', 's', 'a', 'ball_land_x', 'ball_land_y',
            'o_sin', 'o_cos', 'dir_sin', 'dir_cos',
            'player_height_feet', 'player_weight',
            'velocity_x', 'velocity_y', 'acceleration_x', 'acceleration_y',
            'momentum_x', 'momentum_y', 'kinetic_energy',
            'is_offense', 'is_defense', 'is_receiver', 'is_coverage', 'is_passer',
            'distance_to_ball', 'angle_to_ball', 'ball_direction_x', 'ball_direction_y', 'closing_speed',
            'x_lag1', 'y_lag1', 'velocity_x_lag1', 'velocity_y_lag1',
            'x_lag2', 'y_lag2', 'velocity_x_lag2', 'velocity_y_lag2',
            'x_lag3', 'y_lag3', 'velocity_x_lag3', 'velocity_y_lag3',
            'velocity_x_ema', 'velocity_y_ema', 'speed_ema',
            'distance_to_ball_change', 'distance_to_ball_accel', 'time_to_intercept',
            'velocity_alignment', 'velocity_perpendicular', 'accel_alignment',
            'velocity_x_roll3', 'velocity_x_std3', 'velocity_y_roll3', 'velocity_y_std3',
            's_roll3', 's_std3', 'a_roll3', 'a_std3',
            'velocity_x_roll5', 'velocity_x_std5', 'velocity_y_roll5', 'velocity_y_std5',
            's_roll5', 's_std5', 'a_roll5', 'a_std5',
            'velocity_x_roll10', 'velocity_x_std10', 'velocity_y_roll10', 'velocity_y_std10',
            's_roll10', 's_std10', 'a_roll10', 'a_std10',
            'x_lag4', 'y_lag4', 'velocity_x_lag4', 'velocity_y_lag4',
            'x_lag5', 'y_lag5', 'velocity_x_lag5', 'velocity_y_lag5',
            'velocity_x_change', 'velocity_y_change', 'speed_change', 'direction_change',
            'dist_from_sideline', 'dist_from_endzone',
            'receiver_optimality', 'receiver_deviation', 'defender_closing_speed',
            'frames_elapsed', 'normalized_time',
            'distance_to_player_mean_offense', 'distance_to_player_min_offense', 'distance_to_player_max_offense',
            'relative_velocity_magnitude_mean_offense', 'relative_velocity_magnitude_min_offense', 'relative_velocity_magnitude_max_offense',
            'angle_to_player_mean_offense', 'angle_to_player_min_offense', 'angle_to_player_max_offense',
            'distance_to_player_mean_defense', 'distance_to_player_min_defense', 'distance_to_player_max_defense',
            'relative_velocity_magnitude_mean_defense', 'relative_velocity_magnitude_min_defense', 'relative_velocity_magnitude_max_defense',
            'angle_to_player_mean_defense', 'angle_to_player_min_defense', 'angle_to_player_max_defense',
            'nearest_opponent_dist', 'nearest_opponent_angle', 'nearest_opponent_rel_speed',
        ]
        
        # Filter to available columns and fill missing with zeros
        available_features = []
        for feature in feature_cols:
            if feature in input_df.columns:
                available_features.append(feature)
            else:
                input_df[feature] = 0.0
                available_features.append(feature)
        
        print(f"Using {len(available_features)} features")
        
        # Create sequences
        input_df.set_index(['game_id', 'play_id', 'nfl_id'], inplace=True)
        grouped = input_df.groupby(level=['game_id', 'play_id', 'nfl_id'])
        
        target_groups = test_template[['game_id', 'play_id', 'nfl_id']].drop_duplicates()
        
        sequences, sequence_ids = [], []
        
        for _, row in target_groups.iterrows():
            key = (row['game_id'], row['play_id'], row['nfl_id'])
            
            try:
                group_df = grouped.get_group(key)
            except KeyError:
                continue
            
            input_window = group_df.tail(window_size)
            
            if len(input_window) < window_size:
                pad_len = window_size - len(input_window)
                first = input_window.iloc[0:1].copy()
                pad_df = pd.concat([first] * pad_len, ignore_index=True)
                input_window = pd.concat([pad_df, input_window], ignore_index=True)
            
            input_window = input_window.ffill().bfill().fillna(0.0)
            
            seq = input_window[available_features].values
            seq = np.nan_to_num(seq, nan=0.0)
            
            sequences.append(seq)
            sequence_ids.append({
                'game_id': key[0],
                'play_id': key[1],
                'nfl_id': key[2],
                'frame_id': input_window.iloc[-1]['frame_id']
            })
        
        print(f"Created {len(sequences)} sequences")
        return sequences, sequence_ids, available_features

# MAIN PREDICTION PIPELINE

In [ ]:
class MainPredictor:
    def __init__(self):
        self.models = []
        self.scalers = []
        self.config = Config()
        self.feature_engineer = CompleteFeatureEngineer()
        self.loaded = False
    
    def load_pretrained_models(self):
        """Load your LB 0.604 models"""
        if self.loaded:
            return
            
        print("🚀 Loading LB 0.604 pretrained models...")
        
        for fold in range(1, 6):
            model_path = self.config.MODEL_DIR / f"fold_{fold}" / "model.pt"
            scaler_path = self.config.MODEL_DIR / f"fold_{fold}" / "scaler.joblib"
            
            if model_path.exists() and scaler_path.exists():
                try:
                    # Load model
                    checkpoint = torch.load(model_path, map_location='cpu')
                    model = HybridSeqModel(
                        input_dim=114,  # Your models expect 114 features
                        horizon=checkpoint['config']['horizon']
                    )
                    model.load_state_dict(checkpoint['state_dict'])
                    model.to(self.config.DEVICE)
                    model.eval()
                    self.models.append(model)
                    
                    # Load scaler
                    scaler = joblib.load(scaler_path)
                    self.scalers.append(scaler)
                    
                    print(f"✅ Loaded fold {fold}")
                    
                except Exception as e:
                    print(f"❌ Error loading fold {fold}: {e}")
        
        if not self.models:
            raise ValueError("❌ No LB 0.604 models found!")
            
        self.loaded = True
        print(f"🎯 Successfully loaded {len(self.models)} LB 0.604 models")
    
    def predict(self, test: pl.DataFrame, test_input: pl.DataFrame) -> pl.DataFrame:
        """Complete prediction using LB 0.604 pipeline"""
        if not self.loaded:
            self.load_pretrained_models()
        
        print(f"📊 Processing {len(test)} predictions with LB 0.604 pipeline...")
        
        # Convert to pandas for feature engineering
        test_input_pd = test_input.to_pandas()
        test_template_pd = test.to_pandas()
        
        # Prepare sequences with complete feature engineering
        sequences, sequence_ids, feature_cols = self.feature_engineer.prepare_sequences_complete(
            test_input_pd, test_template_pd, is_training=False, window_size=self.config.WINDOW_SIZE
        )
        
        if not sequences:
            print("❌ No sequences prepared - using fallback")
            return pl.DataFrame({'x': [60.0] * len(test), 'y': [26.65] * len(test)})
        
        print(f"🎯 Generated {len(sequences)} sequences with {len(feature_cols)} features")
        
        # Convert to tensor
        X_test = np.stack([seq.astype(np.float32) for seq in sequences])
        
        # Ensemble predictions
        all_dx, all_dy = [], []
        
        for i, (model, scaler) in enumerate(zip(self.models, self.scalers)):
            print(f"🔮 Running LB 0.604 fold {i+1}...")
            
            # Scale features
            X_scaled = scaler.transform(X_test.reshape(-1, X_test.shape[-1])).reshape(X_test.shape)
            X_tensor = torch.from_numpy(X_scaled).to(self.config.DEVICE)
            
            # Predict
            with torch.no_grad():
                dx, dy = model(X_tensor)
                all_dx.append(dx.cpu().numpy())
                all_dy.append(dy.cpu().numpy())
        
        # Ensemble average
        ens_dx = np.mean(np.stack(all_dx), axis=0)
        ens_dy = np.mean(np.stack(all_dy), axis=0)
        
        # Create predictions
        predictions = []
        sequence_df = pd.DataFrame(sequence_ids)
        
        for i, seq_info in sequence_df.iterrows():
            game_id = seq_info['game_id']
            play_id = seq_info['play_id'] 
            nfl_id = seq_info['nfl_id']
            
            # Get target frames
            target_rows = test.filter(
                (pl.col('game_id') == game_id) & 
                (pl.col('play_id') == play_id) & 
                (pl.col('nfl_id') == nfl_id)
            )
            
            if len(target_rows) == 0:
                continue
            
            # Get last position
            last_frame = test_input.filter(
                (pl.col('game_id') == game_id) & 
                (pl.col('play_id') == play_id) & 
                (pl.col('nfl_id') == nfl_id)
            ).sort('frame_id').tail(1)
            
            if len(last_frame) == 0:
                continue
                
            last_x = last_frame['x'][0]
            last_y = last_frame['y'][0]
            
            # Create predictions for each future frame
            target_frames = target_rows.sort('frame_id')
            
            for t in range(len(target_frames)):
                pred_idx = min(t, ens_dx.shape[1] - 1)
                
                pred_x = float(last_x + ens_dx[i, pred_idx])
                pred_y = float(last_y + ens_dy[i, pred_idx])
                
                # Clip to field boundaries
                pred_x = max(self.config.FIELD_X_MIN, min(self.config.FIELD_X_MAX, pred_x))
                pred_y = max(self.config.FIELD_Y_MIN, min(self.config.FIELD_Y_MAX, pred_y))
                
                predictions.append({
                    'x': pred_x,
                    'y': pred_y
                })
        
        print(f"✅ Generated {len(predictions)} predictions using LB 0.604 pipeline")
        
        # Ensure correct count
        if len(predictions) != len(test):
            print(f"⚠️  Count mismatch: {len(predictions)} vs {len(test)}")
            while len(predictions) < len(test):
                predictions.append({'x': 60.0, 'y': 26.65})
            predictions = predictions[:len(test)]
        
        return pl.DataFrame(predictions)

# SERVER STARTUP
## COMPETITION INTERFACE

In [ ]:
lb_predictor = MainPredictor()

def predict(test: pl.DataFrame, test_input: pl.DataFrame) -> pl.DataFrame:
    """Competition prediction function using LB 0.604 pipeline"""
    return lb_predictor.predict(test, test_input)

# SERVER SETUP
print("🚀 Setting up NFL Big Data Bowl 2026 Inference Server...")
print(f"📁 Model directory: {Config.MODEL_DIR}")
print(f"🎯 Target: LB 0.604 Performance")
print(f"🔧 Features: 114 complete features with player interactions")
print(f"🏈 Model: 5-fold ensemble with full feature engineering")

inference_server = kaggle_evaluation.nfl_inference_server.NFLInferenceServer(predict)

if os.getenv('KAGGLE_IS_COMPETITION_RERUN'):
    print("🏈 Starting competition inference server with LB 0.604 pipeline...")
    inference_server.serve()
else:
    print("🔬 Running local test gateway with LB 0.604 pipeline...")
    inference_server.run_local_gateway(('/kaggle/input/nfl-big-data-bowl-2026-prediction/',))